In [47]:
import sys
import os                           # for os interface
import csv
import math
import pickle                       # for full-precision data storage
import numpy as np
import pandas as pd
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from collections import OrderedDict, defaultdict  # append list to disc
import timeit

In [48]:
start_time = timeit.default_timer()
data_file = '../input/itcont.txt'
#test_data = '../input/test_data.txt'
header_file = '../input/indiv_header_file.csv'
elapsed = timeit.default_timer() - start_time
print (elapsed)

5.8689998695626855e-05


In [49]:
start_time = timeit.default_timer()
# read header line
indiv_header=[]
with open(header_file) as f:
    for line in f:
        header_ele=[ele.strip() for ele in line.split(',') ]
        indiv_header.extend(header_ele) # use extend instead of append
elapsed = timeit.default_timer() - start_time
print (elapsed)
dataset_raw = pd.read_csv(data_file,delimiter='|',\
                      names=indiv_header,keep_default_na=False,dtype=str)
elapsed = timeit.default_timer() - start_time
print (elapsed)

0.0016838729934534058
0.06221933900087606


In [50]:
start_time = timeit.default_timer()
# keep columns of interest
dataset_valid=pd.DataFrame()
dataset_valid=dataset_raw.drop(['AMNDT_IND', 'RPT_TP','TRANSACTION_PGI',\
              'IMAGE_NUM','TRANSACTION_TP','ENTITY_TP',\
              'CITY','STATE','EMPLOYER','OCCUPATION','TRAN_ID',
              'FILE_NUM','MEMO_CD','MEMO_TEXT','SUB_ID'], axis=1)

dataset_valid=dataset_valid[dataset_valid['OTHER_ID'].notnull()]
dataset_valid=dataset_valid[dataset_valid.notnull()]
elapsed = timeit.default_timer() - start_time
print (elapsed)

0.012196771989692934


In [51]:
start_time = timeit.default_timer()
# take the first five characters of 'ZIP_CODE'
# dataset_valid['ZIP_CODE']=dataset_valid['ZIP_CODE'].astype(str).str[0:5]
# 'ZIP_CODE' is valid only when the resulting string length is 5
dataset_valid=dataset_valid[dataset_valid['ZIP_CODE'].str.len()>=5]

# take integer in 'TRANSACTION_AMT'
dataset_valid['TRANSACTION_AMT']=dataset_valid['TRANSACTION_AMT'].astype(int)
# 'TRANSACTION_AMT' is valid only when > 0
dataset_valid=dataset_valid[dataset_valid['TRANSACTION_AMT']>5]

In [63]:
# Keep all duplicates
dataset_valid=dataset_valid[dataset_valid.duplicated\
                             (['NAME','ZIP_CODE'],keep=False)]

# date order
donate_time=pd.to_datetime(dataset_valid['TRANSACTION_DT'], format='%m%d%Y', errors='ignore')
dataset_valid['newcol1'] = donate_time.shift() - donate_time
dataset_valid['newcol1'] = dataset_valid['newcol1'].fillna(0)
dataset_valid['newcol2'] = pd.to_timedelta(dataset_valid['newcol1'],errors='coerce').astype(str).str[0:2]

# identity=ID+ZIP
dataset_year=pd.DataFrame()
dataset_year['Identity'] = dataset_valid['CMTE_ID'] + dataset_valid.ZIP_CODE.str[0:5]
dataset_year['TRANS_AMT'] = dataset_valid.TRANSACTION_AMT
dataset_year['index1'] = dataset_valid.index
dataset_year['date'] = pd.to_datetime(dataset_valid['TRANSACTION_DT'], format='%m%d%Y', errors='ignore')

#dataset_year.sort_values(by=['Identity','index1'])

In [64]:
grouped=dataset_year.groupby('Identity')
dataset_year['date_shifted'] = dataset_year.groupby(['Identity'])['date'].shift(1)
dataset_year['date_diff'] = dataset_year['date_shifted'] - dataset_year['date']
dataset_year['date_diff']=pd.to_timedelta(dataset_year['date_diff']).astype(str).str[0:2]
dataset_year=dataset_year.dropna( how='any')
# positives are out of order
dataset_year=dataset_year[dataset_year['date_diff'].astype(int)<0]


#pd.to_datetime(dataset_year['date_shifted'], format='%m%d%Y', errors='ignore')
#for group in grouped:
    # transforming to tuples so that it can be used as keys on a dict
#    lines = [tuple(y) for y in group[1].loc[:,'newcol2'].values.tolist()] 
#    group[group.columns.difference(['newcol2'])]
    #key = tuple(lines) 
    
    
#dataset_valid=dataset_valid[dataset_valid['newcol2'].astype(int)<0]
#grouped=dataset_arrange.groupby('Identity')
#dataset_valid.groupby('NAME')
dataset_year.iloc[0:5]

,Identity,TRANS_AMT,index1,date,date_shifted,date_diff
109,C0039497344425,396,109,2017-01-10,2017-01-06,-4
110,C0039497344425,25,110,2017-01-26,2017-01-10,-1
124,C0010408317050,190,124,2017-01-18,2017-01-04,-1
208,C0061043699507,500,208,2017-01-13,2016-11-30,-4
215,C0016141435243,125,215,2017-01-31,2017-01-13,-1


In [62]:


# first : Mark duplicates as True except for the first occurrence.
#dataset_repeat=dataset_valid[dataset_valid.duplicated\
#                             (['NAME','ZIP_CODE'],keep='first')]

#elapsed = timeit.default_timer() - start_time
#print (elapsed)

In [24]:
dataset_valid.iloc[0:10]

,CMTE_ID,NAME,ZIP_CODE,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,newcol1
0,C00629618,"PEREZ, JOHN A",90017,01032017,40,H6CA34245,NaT
1,C00177436,"DEEHAN, WILLIAM N",300047357,01312017,384,,-28 days
2,C00177436,"WATJEN, THOMAS R.",330375267,01042017,5000,,27 days
3,C00177436,"SABOURIN, JAMES",307502818,01312017,230,,-27 days
4,C00177436,"MAKER, SCOTT T.",040976952,01312017,384,,0 days
5,C00177436,"PYNE, CHRISTOPHER W",020251500,01312017,230,,0 days
6,C00177436,"JEROME, CHRISTOPHER",041051896,01312017,384,,0 days
7,C00177436,"FOLEY, JOSEPH",041051935,01312017,384,,0 days
8,C00177436,"MCGARRY, JOHN",040424132,01312017,384,,0 days
9,C00177436,"SIMONDS, MICHAEL Q",041051972,01312017,384,,0 days


In [74]:
start_time = timeit.default_timer()
dataset_arrange=pd.DataFrame()
dataset_arrange['Identity'] = dataset_repeat['CMTE_ID'] + dataset_repeat.ZIP_CODE.str[0:5] + dataset_repeat.TRANSACTION_DT.str[-4:]
dataset_arrange['TRANS_AMT'] = dataset_repeat.TRANSACTION_AMT
dataset_arrange['index1'] = dataset_arrange.index
elapsed = timeit.default_timer() - start_time
print (elapsed)

0.013398808994679712


In [76]:
dataset_arrange.iloc[0:5]

,Identity,TRANS_AMT,index1
109,C00394973444252017,396,109
110,C00394973444252017,25,110
124,C00104083170502017,190,124
208,C00610436995072017,500,208
215,C00161414352432017,125,215


In [71]:

grouped=dataset_arrange.groupby('Identity')
#grouped.groups['C00394973444252017']
#pd.rolling_quantile(grouped['TRANS_AMT'], 2, 0.3)
grouped['TRANS_AMT'].expanding(min_periods=1,center=False).quantile(quantile=0.3,axis=0,numeric_only=False,interpolation='nearest')
#grouped['TRANS_AMT'].expanding(1).quantile(quantile=0.3,interpolation='nearest')
#total=grouped['TRANS_AMT'].expanding(1).sum()
#times=grouped['TRANS_AMT'].expanding(1).count()
#def percentile(group):
#    s1=group.expanding(1)
#    return np.percentile(s1, 50)
def f1(group):
    return pd.DataFrame({'Perc': group['TRANS_AMT'].expanding(min_periods=1, center=False).quantile(quantile=0.3)})


def f2(group):
    return pd.DataFrame({'Perc': group['TRANS_AMT'].expanding(min_periods=1, center=False)})

#Results = grouped.apply(f)

def rank(array):
    s = pd.Series(array)
    return s.rank(ascending=False)[len(s)-1]

#grouped['TRANS_AMT'].apply(dataset_arrange['TRANS_AMT'], 3, rank)
pd.rolling_apply(grouped['TRANS_AMT'], 3, rank)
#grouped.['TRANS_AMT'].expanding(min_periods=1,center=False).apply
#s1=grouped['TRANS_AMT'].expanding(min_periods=1,center=False)

#s1.series.expand()
#def func_sum(group):
#    return pd.DataFrame({'sum': group.expanding(1).sum(),'times':group.expanding(1).count()} )
#def func_perc(group):
#    return pd.Series(group.expanding(min_periods=1).apply(func=np.percentile(a= q=30)))

#a.rolling(row).apply(func=np.percentile, args=(25,)).tail(1)
#def func_total(group):
#    return pd.Series(group.expanding(1).sum())
#def func_times(group):
#    return pd.Series(group.expanding(1).count())

#grouped['TRANS_AMT'].transform(func_perc)



/Users/YueQian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: FutureWarning: pd.rolling_apply is deprecated for SeriesGroupBy and will be removed in a future version, replace with 
	SeriesGroupBy.rolling(window=3,center=False).apply(func=<function>,args=<tuple>,kwargs=<dict>)


Identity               
C00033555317922017  519    NaN
C00035006945832017  615    NaN
                    618    NaN
                    620    2.0
                    622    2.0
                    624    2.0
                    626    2.0
                    628    2.0
                    630    2.0
                    632    2.0
                    634    2.0
                    636    2.0
                    640    2.0
                    641    2.0
                    643    2.0
                    645    2.0
                    648    2.0
                    649    2.0
                    652    2.0
                    654    2.0
                    655    2.0
                    657    2.0
                    660    2.0
                    662    2.0
                    666    2.0
                    667    2.0
                    668    2.0
                    673    2.0
                    674    2.0
                    676    2.0
                          ... 
C00193433112182

In [48]:
#grouped=dataset_arrange.groupby('Identity')['TRANS_AMT']
#grouped.expanding(min_periods=1).quantile(quantile=0.3,interpolation='nearest')

In [58]:
X

,Perc
109,396.0
110,136.3
124,190.0
208,500.0
215,125.0
217,174.0
219,223.0
221,375.0
223,213.2
225,450.0


In [152]:
start_time = timeit.default_timer()

dd = defaultdict(list)
dataset_final = dataset_arrange.to_dict('records', into=dd)

d1 = defaultdict(list)
d2 = defaultdict(list)
perc = 30.
percent = perc/100.



for item in dataset_final:
    identity=item['Identity']
    amount=item['TRANS_AMT']
    index1=item['index1']
    d1[identity].append(amount)
    if len(d1[identity]) ==1:
        perc_of_cont = amount
        total_cont = amount
        time_of_cont =1
        continue
    else:
        ranked = sorted(d1[identity])
        n = int(round(percent * len(d1[identity])))
        perc_of_cont=ranked[n-1]
        total_cont=sum(d1[identity])
        time_of_cont=len(d1[identity])
    

    d2[index1].append(identity)
    d2[index1].append(perc_of_cont)
    d2[index1].append(total_cont)
    d2[index1].append(time_of_cont)    
    
elapsed = timeit.default_timer() - start_time
print (elapsed)

    
#for k, v in d1.items():       
#    print (k,v)

944.451614868005


In [153]:
#%timeit dataset_final.items()

In [154]:
# group by and sum
# https://stackoverflow.com/questions/33823091/python-pandas-counting-the-frequency-of-a-specific-value-in-each-row-of-datafra
# https://stackoverflow.com/questions/42446006/pandas-find-previous-row-of-matching-value
#data_output['TRANS_AMT_PRE'] = \
#data_output.groupby(['CMTE_ID','ZIP_CODE','TRANS_DT'])['TRANS_AMT'].shift()
#data_output['TRANS_AMT'] + \
#data_output.groupby('ZIP_CODE')['TRANS_AMT'].shift()

In [155]:
#grouped=data_output.groupby(['CMTE_ID','ZIP_CODE','TRANS_DT'])

In [156]:
#new=data_output.groupby(['CMTE_ID']).get_group('C00035006')
#grouped.agg({'TRANS_AMT':np.sum,'TRANS_AMT': np.mean})

In [157]:
#for name, group in grouped:
    #print(name)
#    print(group.TRANS_AMT)

In [158]:
#grouped.bfill